In [1]:
import glob
import pandas as pd
import numpy as np
import re
from konlpy.tag import Kkma
from konlpy.utils import pprint
import pickle
import os
import csv

In [2]:
path = './data/'
filenames = glob.glob(path + "/*.csv")
frame = pd.DataFrame()
list_ = []
for file_ in filenames:
    df = pd.read_csv(file_, index_col=None)
    list_.append(df)
frame = pd.concat(list_)

In [3]:
frame.head()

,name,desc,type1,type2,egg_group
0,이상해씨,태어날 때부터 등에 식물의 씨앗이 있어 조금씩 크게 자란다.태어났을 때부터 등에 이...,풀,독,"괴수,식물"
1,이상해풀,꽃봉오리가 등에 붙어 있으며 양분을 흡수해가면 커다란 꽃이 핀다고 한다.등의 꽃봉오...,풀,독,"괴수,식물"
2,이상해꽃,꽃에서 황홀한 향기가 퍼져나가 싸우는 자의 기분을 달래게 한다.태양 에너지를 영양으...,풀,독,"괴수,식물"
3,파이리,태어날 때부터 꼬리의 불꽃이 타오르고 있다. 불꽃이 꺼지면 그 생명이 다하고 만다....,불꽃,NaN,"괴수,드래곤"
4,리자드,꼬리를 휘둘러 상대를 쓰러트리고 날카로운 발톱으로 갈기갈기 찢어버린다.불타는 꼬리를...,불꽃,NaN,"괴수,드래곤"


In [5]:
def _save_pickle(path, data):
    # save pkl
    f = open(path, 'wb')
    pickle.dump(data, f)
    f.close()

In [4]:
# exchange type1 and type2 and make new_type

# convert 비행 and 노말
def Type(type1, type2):
    if type1 == '노말' and type2 == type2:
        return type2
    elif type2 == '비행':
        return '비행'
    else:
        return type1

# convert jsut 비행
def Type2(type1, type2):
    if type2 == '비행':
        return '비행'
    else:
        return type1
    

frame['new_type'] = frame[['type1','type2']].apply(lambda x: Type(x['type1'], x['type2']), axis=1)
frame.head(10)

,name,desc,type1,type2,egg_group,new_type
0,이상해씨,태어날 때부터 등에 식물의 씨앗이 있어 조금씩 크게 자란다.태어났을 때부터 등에 이...,풀,독,"괴수,식물",풀
1,이상해풀,꽃봉오리가 등에 붙어 있으며 양분을 흡수해가면 커다란 꽃이 핀다고 한다.등의 꽃봉오...,풀,독,"괴수,식물",풀
2,이상해꽃,꽃에서 황홀한 향기가 퍼져나가 싸우는 자의 기분을 달래게 한다.태양 에너지를 영양으...,풀,독,"괴수,식물",풀
3,파이리,태어날 때부터 꼬리의 불꽃이 타오르고 있다. 불꽃이 꺼지면 그 생명이 다하고 만다....,불꽃,NaN,"괴수,드래곤",불꽃
4,리자드,꼬리를 휘둘러 상대를 쓰러트리고 날카로운 발톱으로 갈기갈기 찢어버린다.불타는 꼬리를...,불꽃,NaN,"괴수,드래곤",불꽃
5,리자몽,지상 1400미터까지 날개를 사용해 나는 것이 가능하다. 고열의 불꽃을 내뿜는다.암...,불꽃,비행,"괴수,드래곤",비행
6,꼬부기,긴 목을 등껍질 안에 움추릴 때 강한 물대포를 발사한다.태어난 후 등이 부풀어 단단...,물,NaN,"괴수,수중1",물
7,어니부기,애완동물으로서 인기가 높다. 또 털로 감싸진 꼬리는 장수의 상징이다.자주 수중에 숨...,물,NaN,"괴수,수중1",물
8,거북왕,무거운 몸으로 상대를 덮쳐서 기절시킨다. 위기에 처하면 등껍질에 숨는다.등껍질에 분...,물,NaN,"괴수,수중1",물
9,캐터피,푸른 피부로 감싸져 있다. 탈피하여 성장하면 실을 늘어뜨려 번데기로 바꾼다.발은 짧...,벌레,NaN,벌레,벌레


In [6]:
poke_type_list = list(frame['new_type'].value_counts().index)

# 타입, 타입에 따른 전체 포켓몬, 사용되는 포켓몬, 문장수, 제외시킨 포켓몬 수
info_data = pd.DataFrame(columns=('type', 'poke_cnt', 'useful_poke', 'sentence_cnt', 'extract_poke'))

special_data = [] # extract poke_name, sentence # 제외시킨 데이터 저장
type_dict = dict()

for poke_type in poke_type_list:
    new_data = []
    poke_name = []
    etc_poke = []
    print("....devide by {} type poketmon ...".format(poke_type))
    print("poketmon counts....{}".format(len(frame[frame['new_type']==poke_type])))
    
    # get desc by type
    print(".... get desc by {} type....".format(poke_type))
    for desc in frame.loc[frame['new_type']== poke_type, ['name', 'desc']].values.tolist():
        if '(' in desc[1]:
            special_data += desc
            etc_poke.append(desc[0])
        else:
            poke_name.append(desc[0])
            new_data += re.sub(r"[^ㄱ-힣a-zA-Z0-9.]+", ' ', desc[1]).strip().split('.')
            
    # make uniq sentence
    print("....extract uniq sentence....")
    new_data = list(set(new_data))
    new_data.remove('')
    print("Number of sentence .... {}".format(len(new_data)))
    
    # to word to pos
    print("....convert to pos....")
    kkma = Kkma()
    pos_sentence = []
    for sentence in new_data:
        pos_sentence += [kkma.pos(sentence)]
    
    print("....add dict....")
    # add to dict
    type_dict[poke_type] = pos_sentence
    
    
    # make info data
    info_data = info_data.append([{'type': poke_type, 
                                   'poke_cnt': len(frame[frame['new_type']==poke_type]),
                                    'useful_poke': len(poke_name),
                                     'sentence_cnt': len(new_data),
                                     'extract_poke': len(etc_poke)}], ignore_index=True)
    
            
#     break


DATA_PATH = "./data/"

if not os.path.isdir(DATA_PATH):
    os.mkdir(DATA_PATH)
    
_save_pickle(DATA_PATH + 'type_dict.pickle', type_dict)

info_data.to_csv(DATA_PATH + "pk_info_data.csv", index=False, quotechar='"', encoding='utf-8-sig', quoting=csv.QUOTE_NONNUMERIC)


# a = open('type_dict.pickle', 'rb')
# type_dict_load = pickle.load(a)

....devide by 물 type poketmon ...
poketmon counts....108
.... get desc by 물 type....
....extract uniq sentence....
Number of sentence .... 1453
....convert to pos....
....add dict....
....devide by 비행 type poketmon ...
poketmon counts....98
.... get desc by 비행 type....
....extract uniq sentence....
Number of sentence .... 1234
....convert to pos....
....add dict....
....devide by 풀 type poketmon ...
poketmon counts....74
.... get desc by 풀 type....
....extract uniq sentence....
Number of sentence .... 706
....convert to pos....
....add dict....
....devide by 노말 type poketmon ...
poketmon counts....69
.... get desc by 노말 type....
....extract uniq sentence....
Number of sentence .... 937
....convert to pos....
....add dict....
....devide by 벌레 type poketmon ...
poketmon counts....59
.... get desc by 벌레 type....
....extract uniq sentence....
Number of sentence .... 647
....convert to pos....
....add dict....
....devide by 에스퍼 type poketmon ...
poketmon counts....50
.... get desc by 에스퍼 ty